In [39]:
import os
import cv2
from glob import glob
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from albumentations import *

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile
local_zip = '/content/drive/MyDrive/BlastsOnline.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./BlastsOnline')

zip_ref.close()

In [114]:
drive_path = '/content/BlastsOnline/BlastsOnline'

In [115]:
def create_dir(path):
    """Create a directory"""
    if not os.path.exists(path):
        os.mkdir(path)

In [116]:
def load_data(path):
    images = sorted(glob(f"{path}/Images/*.BMP"))
    masks_te = sorted(glob(f"{path}/GT_TE/*.bmp"))
    masks_icm = sorted(glob(f"{path}/GT_ICM/*.bmp"))
    masks_zp = sorted(glob(f"{path}/GT_ZP/*.bmp"))
    return images,masks_te,masks_icm,masks_zp

In [117]:
def load_data_split(path):
    images = sorted(glob(f"{path}/Images/*.BMP"))
    masks_te = sorted(glob(f"{path}/GT_TE/*.bmp"))
    masks_icm = sorted(glob(f"{path}/GT_ICM/*.bmp"))
    masks_zp = sorted(glob(f"{path}/GT_ZP/*.bmp"))
    masks_Blastocoel = sorted(glob(f"{path}/GT_Blastocoel/*.bmp"))
    masks_ROI = sorted(glob(f"{path}/GT_ROI/*.bmp"))
    masks_label = sorted(glob(f"{path}/GT_Label/*.bmp"))
    Full_mask=sorted(glob(f"{path}/GT_MergedMask/*.bmp"))
    print(len(masks_zp), len(masks_icm), len(masks_te), len(images))
    split_size=round(0.1*len(images))
    split_size2=round(0.1*len(images))

    train_i, test_i, train_te, test_te, train_zp, test_zp,train_b,test_b,train_ROI,test_ROI,train_f,test_f,train_l,test_l ,train_icm, test_icm, = train_test_split(images,masks_te,masks_zp,masks_Blastocoel,masks_ROI,Full_mask,masks_label,masks_icm, test_size=split_size, random_state=42)
    train_i, valid_i, train_te, valid_te, train_zp, valid_zp ,train_icm, valid_icm,train_b,valid_b,train_ROI,valid_ROI,train_f,valid_f,train_l,valid_l = train_test_split(train_i,train_te,train_zp,train_icm,train_b,train_ROI,train_f,train_l, test_size=split_size2, random_state=42)

    print("train, valid, test")
    print(len(train_zp), len(valid_zp),len(test_zp))

    return (train_i, test_i, valid_i), (train_zp, test_zp, valid_zp), (train_icm, test_icm, valid_icm), (train_te, test_te, valid_te),(train_ROI, test_ROI, valid_ROI),(train_f, test_f, valid_f),(train_l, test_l, valid_l),(train_b,test_b,valid_b)

In [118]:
def read_this(image_file):

    image_src = cv2.imread(image_file)
    image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)

    return image_src

In [119]:
def GT_blastocoel(maskA,maskB,maskC,mergedmasks):
    #Find the blastocoel GT through using the ICM, ZP and TE

    merge1=cv2.add(maskA,maskB)
    part_merged=cv2.add(merge1,maskC)
    GT_C=mergedmasks+part_merged
    GT_C[GT_C<255]=0

    return GT_C

In [120]:
def addmasks(maskA, maskB, maskC):
    # FullMask
    merge1 = cv2.add(maskA, maskB)
    merged = cv2.add(merge1, maskC)
    merged = 255 - merged
    th, im_th = cv2.threshold(merged, 220, 255, cv2.THRESH_BINARY_INV);
    im_floodfill = im_th.copy()
    h, w = im_th.shape[:2]
    mask = np.zeros((h + 2, w + 2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0, 0), 255);
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    result = im_th | im_floodfill_inv
    merged = result
    return merged

In [121]:
def generate_mask(zp, icm, te):
  z=zp.copy()
  i=icm.copy()
  t=te.copy()
  z[z == 255] = 32
  i[i == 255] = 64
  t[t == 255] = 128
  merge1 = cv2.add(z, i)
  merged = cv2.add(merge1, t)
  merged = 255 - merged
  return merged

In [123]:
import csv
from google.colab.patches import cv2_imshow

In [125]:
import os
import cv2
from tqdm import tqdm
from albumentations import *

def augment_data(images, new_path, train):
    """Perform data augmentation and save augmented images"""
    print("here!")
    for image in images:
        # Load the image
        image_name = image.split("/")[-1].split(".")[0]

        image = read_this(image)


        # Initialize a list to store augmented images
        augmented_images = [image]

        if train:
            # Define augmentation transforms for training data
            aug = Compose([
                Transpose(1)

            ])
            augmented = aug(image=image)
            augmented_images.append(augmented['image'])
            aug1=Compose([

                HorizontalFlip(p=1),

            ])
            augmented1 = aug1(image=image)
            augmented_images.append(augmented1['image'])
            aug2=Compose([

                VerticalFlip(1),

            ])
            augmented2 = aug(image=image)
            augmented_images.append(augmented2['image'])
            aug3=Compose([

                HorizontalFlip(p=1)
            ])
            augmented3 = aug(image=image)
            augmented_images.append(augmented3['image'])

        # Save the augmented images
        for index, augmented_image in enumerate(augmented_images):
            if train:
                new_image_name = f"{image_name}_aug{index}.bmp"
            else:
                new_image_name = f"{image_name}.bmp"
            new_image_path = os.path.join(new_path, new_image_name)
            cv2.imwrite(new_image_path, augmented_image)

    print("Done!")


In [126]:
def create_dir_for_result(path):
   create_dir(path+"images/")
   create_dir(path+"GT_ZP/")
   create_dir(path+"GT_ICM/")
   create_dir(path+"GT_full/")
   create_dir(path+"label/")
   create_dir(path+"ROI/")
   create_dir(path+"GT_blastocoel/")

In [127]:
def  read(i,zp,icm,te):
      i = read_this(i)
      zp = read_this(zp)
      icm = read_this(icm)
      te = read_this(te)

      mergedmask = addmasks(zp, icm, te)
      blastocoel = GT_blastocoel(te,zp,icm,mergedmask)
      label_mask = generate_mask(zp, icm, te)
      return mergedmask , blastocoel, label_mask


In [128]:
#Load the dataset
(train_i, test_i, valid_i), (train_zp, test_zp, valid_zp), (train_icm, test_icm, valid_icm), (train_te, test_te, valid_te),(train_ROI, test_ROI, valid_ROI),(train_f, test_f, valid_f),(train_l, test_l, valid_l),(train_b,test_b,valid_b)= load_data_split(drive_path)

create_dir(drive_path+"/new_result/")

test_path = drive_path+"/new_result/test/"
valid_path= drive_path+"/new_result/valid/"
train_path= drive_path+"/new_result/train/"

create_dir(train_path)
create_dir(test_path)
create_dir(valid_path)
create_dir_for_result(train_path)
create_dir_for_result(test_path)
create_dir_for_result(valid_path)





249 249 249 249
train, valid, test
199 25 25


In [129]:
label="/content/BlastsOnline/BlastsOnline/GT_Label"
MM="/content/BlastsOnline/BlastsOnline/GT_MergedMask"
b="/content/BlastsOnline/BlastsOnline/GT_Blastocoel"
R="/content/BlastsOnline/BlastsOnline/GT_ROI"
images,masks_te,masks_icm,masks_zp=load_data(drive_path)
for idx, (i, zp, icm, te) in tqdm(enumerate(zip(images,masks_te,masks_icm,masks_zp)), total=len(images)):
    mergedmask,blastocoel,label_mask=read(i, zp, icm, te)
    image_name = i.split("/")[-1].split(".")[0]
    i=read_this(i)
    ROI=i*mergedmask
    ROI_name=f"{image_name}_ROI_{idx}.bmp"
    label_name = f"{image_name}_label_{idx}.bmp"
    MegedMask_name = f"{image_name}_MergedMask_{idx}.bmp"
    b_name = f"{image_name}_blastocoel_{idx}.bmp"
    label_path = os.path.join(label, label_name)
    cv2.imwrite(label_path, label_mask)
    ROI_path = os.path.join(R, ROI_name)
    cv2.imwrite(ROI_path, ROI)
    MM_path = os.path.join(MM, MegedMask_name)
    cv2.imwrite(MM_path, label_mask)
    b_path = os.path.join(b, b_name)
    cv2.imwrite(b_path,blastocoel )


100%|██████████| 249/249 [00:01<00:00, 202.50it/s]


In [130]:
augment_data(valid_ROI,os.path.join(valid_path,"ROI/"),train=True)
augment_data(valid_l,os.path.join(valid_path,"label/"),train=True)
augment_data(valid_b,os.path.join(valid_path,"GT_blastocoel/"),train=True)
augment_data(valid_f,os.path.join(valid_path,"GT_full/"),train=True)
augment_data(valid_zp,os.path.join(valid_path,"GT_ZP/"), train=True)
augment_data(valid_icm,os.path.join(test_path,"GT_ICM/"), train=True)
augment_data(valid_te,os.path.join(valid_path,"GT_TE/"), train=True)


here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!


In [131]:
augment_data(test_ROI,os.path.join(test_path,"ROI/"),train=True)
augment_data(test_l,os.path.join(test_path,"label/"),train=True)
augment_data(test_b,os.path.join(test_path,"GT_blastocoel/"),train=True)
augment_data(test_f,os.path.join(test_path,"GT_full/"),train=True)
augment_data(test_zp,os.path.join(test_path,"GT_ZP/"), train=True)
augment_data(test_icm,os.path.join(test_path,"GT_ICM/"), train=True)
augment_data(test_te,os.path.join(test_path,"GT_TE/"), train=True)

here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!


In [132]:
augment_data(train_ROI,os.path.join(train_path,"ROI/"),train=True)
augment_data(train_l,os.path.join(train_path,"label/"),train=True)
augment_data(train_b,os.path.join(train_path,"GT_blastocoel/"),train=True)
augment_data(train_f,os.path.join(train_path,"GT_full/"),train=True)
augment_data(train_zp,os.path.join(train_path,"GT_ZP/"), train=True)
augment_data(train_icm,os.path.join(train_path,"GT_ICM/"), train=True)
augment_data(train_te,os.path.join(train_path,"GT_TE/"), train=True)


here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!
here!
Done!
